In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
#%pip install -U trl

In [47]:
pip install trl==0.12.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.16.1
    Uninstalling trl-0.16.1:
      Successfully uninstalled trl-0.16.1


In [2]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from sklearn.model_selection import train_test_split

In [3]:
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/'CS 7650'

df = pd.read_excel('reddit_data.xlsx')

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CS 7650
Number of training sentences: 24,057



,ID,Title,Text,Date,Label,Is_MH
21690,nu6qn7,I wish I could have a surgery to get rid of th...,I've done so much work to get where I am and i...,2021-06-07 07:34:48,anxiety,Yes
6698,1fnf28l,Janna Shousha on tiktok,Does anybody know her/ know if she is getting ...,2024-09-23 07:45:13,anorexia,Yes
4092,1f5f6qu,Do you have other health conditions that were ...,"Hi, I'm new here. I started with a new therapi...",2024-08-31 04:38:59,ptsd,Yes
4054,1f51p7y,Distance between sun and earth mentioned in 16...,There is a poem I was reading based on hindu m...,2024-08-30 17:58:22,none,No
21198,md0i1o,I DID IT GUYS! Today marks my ONE MONTH of not...,Woo hoo!! I'm overjoyed! I've officially compl...,2021-03-25 15:12:47,OCD,Yes
4890,1fdlmj6,This disorder has taken everything from me,"My teeth are rotting from the inside out, can ...",2024-09-10 16:08:00,anorexia,Yes
5072,1ffg9v0,Are there any animals associated with space? S...,NaN,2024-09-12 23:14:49,none,No
8019,1fqtge7,TW. Will it EVERE end?,Guys I'm venting sorry for too many posts but ...,2024-09-27 17:39:38,OCD,Yes
10419,1fth0ov,The real reason Luffy and Kidd were put in prison,"I know my art’s not great, but I realized only...",2024-10-01 05:49:51,none,No
20178,ju6r3u,Sub for you.,r/maskdown. It’s a sub for people who mask to ...,2020-11-14 18:35:39,anxiety,Yes


In [5]:
df.shape

(24057, 6)

In [6]:
df['Title'] = df['Title'].fillna('')
df['Text'] = df['Text'].fillna('')
df['Full_Text'] = df['Title'] + ' ' + df['Text']
df['Full_Text'] = df['Full_Text'].astype(str)
df['Label'] = np.where(df['Label'] == 'anorexia', 'eating disorder', df['Label'])
df_subset1 = df[['ID', 'Full_Text','Label']]
df_subset1 = df_subset1[0:5000]
df_subset1 = df_subset1.rename(columns={'Full_Text' : 'text'})
df_subset1 = df_subset1.rename(columns={'text' : 'statement', 'Label': 'status'})
df_subset1.head()

,ID,statement,status
0,100jk9m,The Person Who Traumatized You DMed you on Ins...,ptsd
1,100t81n,PTSD from antibiotics I took an antibiotic cal...,ptsd
2,100xfwl,guys i made a phone call!!!!!!!! basically my ...,anxiety
3,100y8dh,The Time when I was Sexually Harassed online (...,ptsd
4,101kgaa,Having a panic attack due to real event. need ...,OCD


In [7]:
df_subset1.shape

(5000, 3)

In [8]:
#Create a function to clean text in the datafrmae
def clean_text(text):

  # Remove emojis
  text = re.sub(r'[:;][\w\d-]*[:;]', '', text)

  # Remove emoticons
  text = re.sub(r'\([^\)]*\)', '', text)

  # Remove hyperlinks
  text = re.sub(r'https?://[^\s]+', '', text)

  # Remove non-English characters
  text = re.sub(r'[^\w\s]', '', text)

  # Convert all text into lowercase
  text = text.lower()

  return text

In [9]:
import re
df_subset1['statement'] = df_subset1['statement'].apply(clean_text)
df_subset1.sample(10)

,ID,statement,status
2868,1dut6ru,the final of the 2030 fifa world cup will take...,none
76,10edoqt,does any delusion like that exist coming from ...,schizophrenia
2496,1d024nz,anyone else lose their gallbladder to restrict...,eating disorder
1884,1apuujl,is it weird being friends with a 15 years old ...,OCD
3922,1f3rbzp,my experience thus far with ptsd sooo like a...,ptsd
1333,17io9tl,saucy arr of god i am the saucy arr of god\n\n...,schizophrenia
2063,1bc60w4,ocd and micromanagement im ocd and have worked...,OCD
4957,1fecm52,i despise doctors i am currently in stable r...,eating disorder
3486,1ex1dky,is revenge the only option for ptsd victims i ...,ptsd
3507,1exd5oq,visit ara the celestial altar,none


In [10]:
import sklearn
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_subset1, test_size=0.20, random_state=123)

In [11]:
def generate_prompt(data_point):
    return f"""
            Classify the text into none, depression, anxiety, bipolar, ptsd, schizophrenia, eating disorder, or OCD and return the answer as the corresponding mental health disorder label.
text: {data_point["statement"]}
label: {data_point["status"]}""".strip()

def generate_test_prompt(data_point):
    return f"""
            Classify the text into none, depression, anxiety, bipolar, ptsd, schizophrenia, eating disorder, or OCD and return the answer as the corresponding mental health disorder label.
text: {data_point["statement"]}
label: """.strip()

In [12]:
X_train = train_df.copy()
X_test = test_df.copy()
X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)

In [13]:
X_train

,ID,statement,status,text
1863,1amwql5,my therapist says i was traumatized by heartbr...,ptsd,"Classify the text into none, depression, anxie..."
420,129dayq,have you ever seen oh what is that the router ...,schizophrenia,"Classify the text into none, depression, anxie..."
3260,1eorv8v,breaking manchester united have reached an agr...,none,"Classify the text into none, depression, anxie..."
742,142al7q,advice ethics what to do noticing a person wi...,eating disorder,"Classify the text into none, depression, anxie..."
4809,1fco5wm,is this sexual trauma for instance im 21 the o...,ptsd,"Classify the text into none, depression, anxie..."
...,...,...,...,...
1593,18tb47w,i dont get the diet coke hype \n\ni seriously ...,eating disorder,"Classify the text into none, depression, anxie..."
4060,1f5399c,eating nuts advice i dont know if this is tot...,eating disorder,"Classify the text into none, depression, anxie..."
1346,17meeq1,would you rather restriction edition possible...,eating disorder,"Classify the text into none, depression, anxie..."
3454,1ewifx6,recently diagnosed possible trigger warning\n...,eating disorder,"Classify the text into none, depression, anxie..."


In [14]:
y_true = X_test.loc[:,'status']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

In [15]:
X_train.status.value_counts()

,count
status,
eating disorder,1010
ptsd,742
none,684
schizophrenia,538
OCD,323
anxiety,285
depression,246
bipolar,172


In [16]:
y_true.value_counts()

,count
status,
eating disorder,259
ptsd,181
none,180
schizophrenia,123
OCD,85
depression,65
anxiety,65
bipolar,42


In [17]:
train_data = Dataset.from_pandas(X_train[["text"]])
train_data['text'][3]

'Classify the text into none, depression, anxiety, bipolar, ptsd, schizophrenia, eating disorder, or OCD and return the answer as the corresponding mental health disorder label.\ntext: advice ethics  what to do noticing a person with advanced anorexia in strenuous workout class i recently went to a barre class  and there was a woman there who was quite clearly had some eating disorder and it seemed rather advanced extreme seemingly not much muscle extremely skinnyboney very frail \n\nas an onlooker it just didnt seem right that she was in such a strenuous class and it just made me wonder about the ethics of it of course you might argue its also unethical to bar someone from working out but in any case if someone has health issues is deathly ill or even severely pregnant it isnt advised to do certain activities\n\nits just been weighing on my mind lately as im just unsure of what to do in these kinds of instances is it helpful to say something is it destructivetriggering to say anything

In [18]:
midpoint = len(X_test) // 2
eval_data = Dataset.from_pandas(X_test[["text"]][:midpoint])

In [19]:
eval_data[0:5]

{'text': ['Classify the text into none, depression, anxiety, bipolar, ptsd, schizophrenia, eating disorder, or OCD and return the answer as the corresponding mental health disorder label.\ntext: just because we have schizophrenia doesnt mean we cant do anything here is the game i am making \nlabel:',
  'Classify the text into none, depression, anxiety, bipolar, ptsd, schizophrenia, eating disorder, or OCD and return the answer as the corresponding mental health disorder label.\ntext: i feel really awful  on wednesday i had a terrible date with my girlfriend she said some mean things to me at the end of the date and i went manic when i got home and told her i didnt want to be with her anymore \n\ni said some harsh and awful things to her from 9pm to 5am she knows im bipolar so she understood my reaction and tried to fix things between us \n\non thursday we saw each other and talked about the incident she told me she was hurt but that she understood she also told me that she wants to be 

In [20]:
eval_data.shape

(500, 2)

In [22]:
%pip install transformers bitsandbytes

In [23]:
!pip install llama-stack

In [24]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [25]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

In [26]:
pip install -U bitsandbytes

In [27]:
from transformers import AutoModelForCausalLM
from transformers import BitsAndBytesConfig

base_model_name = "meta-llama/Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [28]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token_id = tokenizer.eos_token_id

In [29]:
def predict(test, model, tokenizer):
    y_pred = []
    categories = ["none", "depression", "anxiety", "bipolar", "eating disorder", "OCD", "ptsd", "schizophrenia"]

    for i in tqdm(range(len(test))):
        prompt = test.iloc[i]["text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens=2,
                        temperature=0.1)

        result = pipe(prompt)
        answer = result[0]['generated_text'].split("label:")[-1].strip()

        # Determine the predicted category
        for category in categories:
            if category.lower() in answer.lower():
                y_pred.append(category)
                break
        else:
            y_pred.append("none")

    return y_pred

In [30]:
X_test.shape

(1000, 1)

In [31]:
y_pred = predict(X_test[0:100], model, tokenizer)

100%|██████████| 100/100 [00:40<00:00,  2.48it/s]


In [32]:
def evaluate(y_true, y_pred):
    labels = ["none", "depression", "anxiety", "bipolar", "eating disorder", "OCD", "ptsd", "schizophrenia"]
    mapping = {label: idx for idx, label in enumerate(labels)}

    def map_func(x):
        return mapping.get(x, -1)

    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)


    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    print(f'Accuracy: {accuracy:.3f}')


    unique_labels = set(y_true_mapped)

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped)) if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {labels[label]}: {label_accuracy:.3f}')


    class_report = classification_report(y_true=y_true_mapped, y_pred=y_pred_mapped, target_names=labels, labels=list(range(len(labels))))
    print('\nClassification Report:')
    print(class_report)

    conf_matrix = confusion_matrix(y_true=y_true_mapped, y_pred=y_pred_mapped, labels=list(range(len(labels))))
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [33]:
#Results of model before finetuning
evaluate(y_true[0:100], y_pred)

Accuracy: 0.680
Accuracy for label none: 1.000
Accuracy for label depression: 0.600
Accuracy for label anxiety: 0.800
Accuracy for label bipolar: 0.750
Accuracy for label eating disorder: 0.875
Accuracy for label OCD: 0.571
Accuracy for label ptsd: 0.429
Accuracy for label schizophrenia: 0.400

Classification Report:
                 precision    recall  f1-score   support

           none       0.45      1.00      0.62         9
     depression       0.19      0.60      0.29         5
        anxiety       0.36      0.80      0.50         5
        bipolar       1.00      0.75      0.86         4
eating disorder       1.00      0.88      0.93        32
            OCD       1.00      0.57      0.73        14
           ptsd       1.00      0.43      0.60        21
  schizophrenia       0.80      0.40      0.53        10

       accuracy                           0.68       100
      macro avg       0.73      0.68      0.63       100
   weighted avg       0.86      0.68      0.71      

In [34]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

modules = find_all_linear_names(model)
modules

['down_proj', 'up_proj', 'q_proj', 'v_proj', 'k_proj', 'gate_proj', 'o_proj']

In [35]:
from trl import SFTTrainer, SFTConfig

In [36]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [37]:
peft_config = LoraConfig(
    lora_alpha=128,
    lora_dropout=0,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules,
)

In [38]:
training_arguments = SFTConfig(
    output_dir= '/outputs/',
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=False,
    lr_scheduler_type="cosine",
    report_to="wandb",
    eval_strategy="steps",
    eval_steps = 0.2
)

In [39]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="text",
    processing_class=tokenizer,
    max_seq_length=256,
    packing=False,
    dataset_kwargs={
    "add_special_tokens": False,
    "append_concat_token": False,
    }
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, dataset_kwargs. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:334: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you pas

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [40]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: prathusha-dinesh97 (prathusha-dinesh97-georgia-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
50,2.373700,2.218991
100,2.131700,2.199017
150,2.246200,2.178530
200,2.081800,2.168076
250,2.036400,2.164224


TrainOutput(global_step=250, training_loss=2.2008307647705077, metrics={'train_runtime': 4930.7708, 'train_samples_per_second': 0.811, 'train_steps_per_second': 0.051, 'total_flos': 2.7680930921373696e+16, 'train_loss': 2.2008307647705077, 'epoch': 1.0})

In [41]:
output_dir= '/outputs/'
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('/outputs/tokenizer_config.json',
 '/outputs/special_tokens_map.json',
 '/outputs/tokenizer.json')

In [42]:
y_pred = predict(X_test[midpoint:], model, tokenizer)
evaluate(y_true[midpoint:], y_pred)

100%|██████████| 500/500 [04:54<00:00,  1.70it/s]

Accuracy: 0.922
Accuracy for label none: 0.981
Accuracy for label depression: 0.769
Accuracy for label anxiety: 0.800
Accuracy for label bipolar: 0.783
Accuracy for label eating disorder: 0.976
Accuracy for label OCD: 0.879
Accuracy for label ptsd: 0.956
Accuracy for label schizophrenia: 0.866

Classification Report:
                 precision    recall  f1-score   support

           none       1.00      0.98      0.99       105
     depression       0.87      0.77      0.82        26
        anxiety       0.83      0.80      0.81        30
        bipolar       0.86      0.78      0.82        23
eating disorder       0.95      0.98      0.96       126
            OCD       0.91      0.88      0.89        33
           ptsd       0.96      0.96      0.96        90
  schizophrenia       0.79      0.87      0.83        67

       accuracy                           0.92       500
      macro avg       0.90      0.88      0.88       500
   weighted avg       0.92      0.92      0.92      

In [43]:
evaluate(y_true[midpoint:], y_pred)

Accuracy: 0.922
Accuracy for label none: 0.981
Accuracy for label depression: 0.769
Accuracy for label anxiety: 0.800
Accuracy for label bipolar: 0.783
Accuracy for label eating disorder: 0.976
Accuracy for label OCD: 0.879
Accuracy for label ptsd: 0.956
Accuracy for label schizophrenia: 0.866

Classification Report:
                 precision    recall  f1-score   support

           none       1.00      0.98      0.99       105
     depression       0.87      0.77      0.82        26
        anxiety       0.83      0.80      0.81        30
        bipolar       0.86      0.78      0.82        23
eating disorder       0.95      0.98      0.96       126
            OCD       0.91      0.88      0.89        33
           ptsd       0.96      0.96      0.96        90
  schizophrenia       0.79      0.87      0.83        67

       accuracy                           0.92       500
      macro avg       0.90      0.88      0.88       500
   weighted avg       0.92      0.92      0.92      

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Get attention weights fo r a specific input
inputs = tokenizer("", return_tensors="pt")

outputs = model(**inputs, output_attentions=True)

# Extract attention weights (for all heads, pick one for visualization)
attention = outputs.attentions[-1]  # Last layer attention
head_attention = attention[8][8  ]  # First example, first head

# Visualize
plt.figure(figsize=(10, 8))
sns.heatmap(head_attention.detach().cpu().numpy(), cmap="coolwarm", xticklabels=inputs.tokens, yticklabels=inputs.tokens)
plt.title("Attention Weights")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Get attention weights fo r a specific input
inputs = tokenizer("no fear of missing out. If I drop dead right now I wouldn't be sad. We live in a sad and boring Period, heading towards dystopia.(total population control and enslavement).", return_tensors="pt").to(model.device) # Move inputs to the same device as the model
outputs = model(**inputs, output_attentions=True)

# Extract attention weights (for all heads, pick one for visualization)
attention = outputs.attentions[-1]  # Last layer attention
head_attention = attention[0][5]  # First example, first head # Changed to [0][0]

# Visualize
plt.figure(figsize=(10, 8))
sns.heatmap(head_attention.detach().cpu().numpy(), cmap="coolwarm", xticklabels=inputs.tokens(), yticklabels=inputs.tokens()) # Changed to inputs.tokens
plt.title("Attention Weights")
plt.show()